# Sequence Models & How to use them

**학습목표**
1. Bidirectional layer를 이해한다.
2. Bidirectional layer를 사용할 줄 안다.

-----------------
* Thanks to : 한기영 대표님 @ Data Insight

## 1.환경 및 데이터 준비

### Import Packages

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_squared_error

### Data Loading

In [2]:
url = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'

# 판다스로 데이터를 불러오시오.
data = pd.read_csv(url)

##3.데이터 준비

### Date 컬럼을 제거하시오.

In [3]:
data.drop('Date', axis=1, inplace=True)
data.head()

,Open,High,Low,Close,Adj_Close,Volume
0,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


## 4.Sequence 데이터 구조로 만들기

**조건**
* 내일의 Close를 예측할 것이다. 이를 Y로 둘 것.
* timestep은 10주를 본다. (주식은 5일이 1주일)
* 맞추어 전처리

**데이터 분할 규칙**
* 가장 최근 1주일을 테스트 데이터로 둔다.
* 테스트 데이터를 제외하고, 가장 최근 2주를 벨리데이션 데이터로 둔다.

In [4]:
# 판다스 데이터 프레임을 넘파이로 옮김.
data = data.values

In [5]:
# 여기에 의미있는 기간(timestep을 지정해 봅시다.)
timestep= 50

x = np.array(
    [data[i: i+timestep, :] for i in range(len(data)- timestep)]
)
y = np.array(
    [data[i+timestep, 3] for i in range(len(data)- timestep)]
)

x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5:]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5:]


## 모양 확인 필수
print(x.shape, y.shape)
print('-'* 50)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(4342, 50, 6) (4342,)
--------------------------------------------------
(4327, 50, 6) (4327,)
(10, 50, 6) (10,)
(5, 50, 6) (5,)


데이터의 구조 : [n, timestep, feature수]

# 같이 해보기
**[참고링크](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 16개 필터 사용할 것
    * window_size(filter_size)는 5일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
5. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어



In [6]:
from tensorflow.keras.layers import Dense, Flatten, SimpleRNN
from tensorflow.keras.layers import Input, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Conv1D, MaxPool1D

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.backend import clear_session

In [16]:
####################
## Your Code Here ##
####################
clear_session()

#input layer
il = Input(shape=(50, 6))



# Conv1D
# 16개 필터 사용할 것
# window_size(filter_size)는 5일
# activation은 swish

#hidden layer
hl = Conv1D(filters=16, kernel_size=5, activation='swish', padding='valid')(il)

# MaxPool1D
# 필터 사이즈(window size)는 2일
hl = MaxPool1D(2)(hl)

# Bidirectional 레이어
# LSTM, 히든스테이트 노드 32개
hl = Bidirectional(layer=LSTM(32, return_sequences=True))(hl)

# Bidirectional 레이어
# LSTM, 히든스테이트 노드 32개
hl = Bidirectional(layer=LSTM(32, return_sequences=True))(hl)

# 플래튼
hl = Flatten()(hl)

#output layer
ol = Dense(1, activation='relu')(hl)

model = Model(il, ol)
model.summary()





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 6)]           0         
                                                                 
 conv1d (Conv1D)             (None, 46, 16)            496       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 16)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 23, 64)           12544     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 23, 64)           24832     
 nal)                                                            
                                                             

In [17]:
# compile
model.compile(loss='mae', optimizer='adam')

In [18]:
## 학습도 시킬 것
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                patience=4,
                                verbose=1,
                                factor=0.2,
                                min_lr = 0.000001)
es = EarlyStopping(monitor='val_loss',
                  min_delta = 0,
                  patience=5,
                  verbose=1,
                  restore_best_weights=True)



model.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_val, y_val)
,callbacks=[lr_reduction, es])

Epoch 1/30
136/136 [==============================] - 9s 20ms/step - loss: 23.4526 - val_loss: 94.3592 - lr: 0.0010
Epoch 2/30
136/136 [==============================] - 1s 10ms/step - loss: 20.8988 - val_loss: 87.2380 - lr: 0.0010
Epoch 3/30
136/136 [==============================] - 1s 9ms/step - loss: 20.9312 - val_loss: 88.0993 - lr: 0.0010
Epoch 4/30
136/136 [==============================] - 1s 9ms/step - loss: 21.2261 - val_loss: 93.8963 - lr: 0.0010
Epoch 5/30
136/136 [==============================] - 1s 9ms/step - loss: 21.0703 - val_loss: 96.5850 - lr: 0.0010
Epoch 6/30
132/136 [============================>.] - ETA: 0s - loss: 21.0370
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
136/136 [==============================] - 1s 9ms/step - loss: 20.9209 - val_loss: 90.1640 - lr: 0.0010
Epoch 7/30
136/136 [==============================] - 1s 9ms/step - loss: 20.4575 - val_loss: 86.3204 - lr: 2.0000e-04
Epoch 8/30
136/136 [=========================

In [19]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_error as MSE
MSE(y_test, y_pred)**0.5

1/1 [==============================] - 1s 1s/step


74.203811677196

## Q1. Conv1D + Bidirectional

**[공식문서 적극 참고](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 32개 필터 사용할 것
    * window_size(filter_size)는 10일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 16개
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 24개
4. 플래튼
6. Fully Connected : 노드 256개, swish
5. 회귀를 위한 적절한 아웃풋 레이어


In [20]:
####################
## Your Code Here ##
####################

# input_layer
il = Input(shape=(50, 6))

# Conv1D
# 32개 필터 사용할 것
# window_size(filter_size)는 10일
# activation은 swish

hl = Conv1D(filters=32, kernel_size=10, padding='valid', activation='swish')(il)

# MaxPool1D
# 필터 사이즈(window size)는 2일
hl = MaxPool1D(2)(hl)

# Bidirectional 레이어
# forward layer : LSTM, 히든스테이트 노드 24개
# backward layer : GRU, 히든스테이트 노드 16개
forward_layer = LSTM(24, return_sequences=True)
backward_layer = GRU(16, return_sequences=True, go_backwards=True)
hl = Bidirectional(layer=forward_layer, backward_layer = backward_layer)(hl)

# Bidirectional 레이어
# forward layer : LSTM, 히든스테이트 노드 24개
# backward layer : GRU, 히든스테이트 노드 24개
forward_layer = LSTM(24, return_sequences=True)
backward_layer = GRU(16, return_sequences=True, go_backwards=True)
hl = Bidirectional(layer=forward_layer, backward_layer = backward_layer)(hl)

# 플래튼
hl = Flatten()(hl)

# Fully Connected : 노드 256개, swish
hl = Dense(256, activation='swish')(hl)

# output layer
ol = Dense(1, activation='relu')(hl)

model = Model(il, ol)

# model compile
model.compile(loss='mae', optimizer='adam')

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 6)]           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 41, 32)            1952      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 20, 32)           0         
 1D)                                                             
                                                                 
 bidirectional_2 (Bidirectio  (None, 20, 40)           7872      
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 20, 40)           9024      
 nal)                                                            
                                                           

In [21]:
## 학습도 시킬 것
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                patience=4,
                                verbose=1,
                                factor=0.2,
                                min_lr = 0.000001)
es = EarlyStopping(monitor='val_loss',
                  min_delta = 0,
                  patience=5,
                  verbose=1,
                  restore_best_weights=True)



model.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_val, y_val)
,callbacks=[lr_reduction, es])

Epoch 1/30
136/136 [==============================] - 9s 19ms/step - loss: 21.8122 - val_loss: 94.7579 - lr: 0.0010
Epoch 2/30
136/136 [==============================] - 1s 9ms/step - loss: 21.3796 - val_loss: 93.6776 - lr: 0.0010
Epoch 3/30
136/136 [==============================] - 1s 9ms/step - loss: 21.4995 - val_loss: 97.3942 - lr: 0.0010
Epoch 4/30
136/136 [==============================] - 1s 9ms/step - loss: 21.5589 - val_loss: 96.3873 - lr: 0.0010
Epoch 5/30
136/136 [==============================] - 1s 9ms/step - loss: 21.4931 - val_loss: 97.7135 - lr: 0.0010
Epoch 6/30
131/136 [===========================>..] - ETA: 0s - loss: 21.4924
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
136/136 [==============================] - 1s 9ms/step - loss: 21.4921 - val_loss: 97.9697 - lr: 0.0010
Epoch 7/30
136/136 [==============================] - 1s 10ms/step - loss: 21.3659 - val_loss: 96.1113 - lr: 2.0000e-04
Epoch 7: early stopping


In [22]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_error as MSE
MSE(y_test, y_pred)**0.5

1/1 [==============================] - 1s 1s/step


93.73187930289737